# Information Retrieval / Suche mit Python

### 1) Keyword-basierte Suche · 2) Gewichtete Suche mit Whoosh · 3) Semantische Suche mit Embeddings

In dieser Übung lernen Sie verschiedene Möglichkeiten kennen, eine Suche in einem kleinen Beispielkorpus zu implementieren - von einem einfachen keyword-basierten Ansatz über die eine gewichtete Suche mit TF-IDF bis hin zu semantischer Suche mit Embeddings. 


**Vorbereitung:**
1. `venv` aktivieren!
2. Benötigte Libraries installieren: `pip install whoosh sentence-transformers numpy`
3. Neu installierte Libraries zu `requirements.txt` hinzufügen: `pip freeze > requirements.txt`

## Beispielkorpus erzeugen

Wir arbeiten mit einer erweiterten Version des FAQ-Korpus aus der Python-Einführung. 

Um diesen zu erzeugen habe ich den ursprünglichen Korpus mit Hilfe eines LLMs [*augmentiert*](https://www.datacamp.com/tutorial/complete-guide-data-augmentation), d.h. Variationen der Beispieldokumente erzeugt, um die Datenbasis zu erweitern: 

In [48]:
documents = [
    'Wie kann ich mein Passwort Passwort zurücksetzen?',
    'Wie ändere ich mein Passwort?',
    'Passwort vergessen - wie kann ich es zurücksetzen?',
    'Wie kann ich mein Login-Passwort erneuern?',
    'Wo finde ich meine Bestellhistorie?',
    'Wie kann ich meine bisherigen Bestellungen einsehen?',
    'Wo sehe ich meine vergangenen Bestellungen?',
    'Wo kann ich meine Bestellungen überprüfen?',
    'Bestellhistorie abrufen wie geht das?',
    'Wie kann ich meine Lieferadresse ändern?',
    'Lieferadresse aktualisieren wie geht das?',
    'Wie ändere ich meine Versandadresse?',
    'Kann ich meine Adresse nachträglich ändern?',
    'Adresse für Lieferung ändern möglich?',
    'Wie kontaktiere ich den Kundendienst?',
    'Wie erreiche ich den Support?',
    'Kundendienst kontaktieren wie?',
    'Wo kann ich den Kundenservice erreichen?',
    'Wie bekomme ich Hilfe vom Support-Team?',
    'Welche Zahlungsmethoden werden akzeptiert?',
    'Welche Bezahlmöglichkeiten gibt es?',
    'Wie kann ich bezahlen?',
    'Akzeptierte Zahlungsmethoden - Übersicht',
    'Welche Zahlungsarten stehen zur Verfügung?',
    'Wie kann ich meine Bestellung stornieren?',
    'Bestellung rückgängig machen wie?',
    'Wie annulliere ich eine Bestellung?',
    'Kann ich meine Bestellung noch stornieren?',
    'Stornierung einer Bestellung Anleitung',
    'Wie lange dauert der Versand?',
    'Versanddauer - wie lange dauert es?',
    'Wann wird meine Bestellung geliefert?',
    'Lieferzeitraum - wie lang ist er?',
    'Wie schnell kommt meine Bestellung an?',
    'Kann ich Artikel nach der Bestellung noch ändern?',
    'Kann ich meine Bestellung nachträglich bearbeiten?',
    'Artikel in einer bereits getätigten Bestellung ändern - geht das?',
    'Bestellung nachträglich ändern - möglich?',
    'Kann ich Produkte nach der Bestellung austauschen?'
]

print(f'{len(documents)} documents loaded.')

39 documents loaded.


### 1. Einen einfachen **Invertierten Index** erzeugen

Bei dieser Aufgabe geht es darum, die Dokument aus dem Korpus so zu organisieren, dass Sie schnell Dokumente finden können, die ein gesuchtes Wort enthalten.

#### 1. Erstellen Sie einen invertierten Index:

- Erzeugen Sie ein dictionary, in dem die keys Wörter und die values Listen mit den IDs der Dokumente sind, in denen das jeweilige Wort vorkommt.

- Preprocessing-Vorgaben: Schreiben Sie alle Wörter in Kleinbuchstaben und entfernen Sie Satzzeichen. 

#### 2. Schreiben Sie eine Funktion, um in Ihrem invertierten Index nach einem Wort zu suchen:

- Die Funktion soll eine Liste mit allen IDs der Dokumente zurückgeben, die das gesuchte Wort enthalten.

- Testen Sie die Funktion, indem Sie jedes passende Dokument und seine ID mit `print()` ausgeben.

#### Beispiel-Ausgabe: 

```
[[0, 'Wie kann ich mein Passwort Passwort zurücksetzen?'],
 [0, 'Wie kann ich mein Passwort Passwort zurücksetzen?'],
 [1, 'Wie ändere ich mein Passwort?'],
 [2, 'Passwort vergessen - wie kann ich es zurücksetzen?']]
```

#### Hinweise:
- Nutzen Sie `.split()` für die Tokenisierung.

- Nutzen Sie `.lower()`,  `.strip('')` und ggf. `.replace()` für das Preprocessing.

- Mit `enumerate(documents)` können Sie sowohl auf das jeweilige Dokument als auch auf den Index zugreifen: 

- Beachten Sie, dass ein Wort in mehreren Dokumenten vorkommen kann - speichern Sie deshalb pro Wort eine Liste mit Document-IDs.

In [49]:
# 1. Invertierten Index erzeugen
# a) Funktion für Preprocessing
def preprocess(text):
    # Kleinbuchstaben, Entfernen von Sonderzeichen
    text = text.lower().replace(",", " ").replace("!", " ").replace("?", " ").replace("-", " ").strip()

    # Tokenisierung
    text = text.split()
    return text
# Testen
for satz in documents: 
    print(preprocess(satz))


['wie', 'kann', 'ich', 'mein', 'passwort', 'passwort', 'zurücksetzen']
['wie', 'ändere', 'ich', 'mein', 'passwort']
['passwort', 'vergessen', 'wie', 'kann', 'ich', 'es', 'zurücksetzen']
['wie', 'kann', 'ich', 'mein', 'login', 'passwort', 'erneuern']
['wo', 'finde', 'ich', 'meine', 'bestellhistorie']
['wie', 'kann', 'ich', 'meine', 'bisherigen', 'bestellungen', 'einsehen']
['wo', 'sehe', 'ich', 'meine', 'vergangenen', 'bestellungen']
['wo', 'kann', 'ich', 'meine', 'bestellungen', 'überprüfen']
['bestellhistorie', 'abrufen', 'wie', 'geht', 'das']
['wie', 'kann', 'ich', 'meine', 'lieferadresse', 'ändern']
['lieferadresse', 'aktualisieren', 'wie', 'geht', 'das']
['wie', 'ändere', 'ich', 'meine', 'versandadresse']
['kann', 'ich', 'meine', 'adresse', 'nachträglich', 'ändern']
['adresse', 'für', 'lieferung', 'ändern', 'möglich']
['wie', 'kontaktiere', 'ich', 'den', 'kundendienst']
['wie', 'erreiche', 'ich', 'den', 'support']
['kundendienst', 'kontaktieren', 'wie']
['wo', 'kann', 'ich', 'den',

In [50]:
# b) Invertierten Index bauen
#ein leeres Dictionary für den invertierten Index initialisieren
inverted_index = {}
doc_id = 0
# Dokumente durchlaufen und den Index füllen
for doc_id, text in enumerate(documents): # doc_id wird durch enumerate automatisch erhöht
    # Preprocessing
    words = preprocess(text)
    # Wörter zum Index hinzufügen
    for word in words:
        # Wenn das Wort noch nicht im Index ist, füge es hinzu
        if word not in inverted_index:
            inverted_index[word] = []
            # Füge die Dokument-ID zur Liste der Dokumente für dieses Wort hinzu
        inverted_index[word].append(doc_id)

# Testen
for word, doc_ids in inverted_index.items():
    print(f"'{word}': {doc_ids}")

'wie': [0, 1, 2, 3, 5, 8, 9, 10, 11, 14, 15, 16, 18, 21, 24, 25, 26, 29, 30, 32, 33]
'kann': [0, 2, 3, 5, 7, 9, 12, 17, 21, 24, 27, 34, 35, 38]
'ich': [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 14, 15, 17, 18, 21, 24, 26, 27, 34, 35, 38]
'mein': [0, 1, 3]
'passwort': [0, 0, 1, 2, 3]
'zurücksetzen': [0, 2]
'ändere': [1, 11]
'vergessen': [2]
'es': [2, 20, 30]
'login': [3]
'erneuern': [3]
'wo': [4, 6, 7, 17]
'finde': [4]
'meine': [4, 5, 6, 7, 9, 11, 12, 24, 27, 31, 33, 35]
'bestellhistorie': [4, 8]
'bisherigen': [5]
'bestellungen': [5, 6, 7]
'einsehen': [5]
'sehe': [6]
'vergangenen': [6]
'überprüfen': [7]
'abrufen': [8]
'geht': [8, 10, 36]
'das': [8, 10, 36]
'lieferadresse': [9, 10]
'ändern': [9, 12, 13, 34, 36, 37]
'aktualisieren': [10]
'versandadresse': [11]
'adresse': [12, 13]
'nachträglich': [12, 35, 37]
'für': [13]
'lieferung': [13]
'möglich': [13, 37]
'kontaktiere': [14]
'den': [14, 15, 17]
'kundendienst': [14, 16]
'erreiche': [15]
'support': [15, 18]
'kontaktieren': [16]
'kundenservice': 

In [51]:
# 2. Such-Funktion
def search (query):
    # Preprocessing der Suchanfrage
    query_words = preprocess(query)
    # Gefundene Dokumente initialisieren
    found_docs = set()
    for word in query_words:
        if word in inverted_index:
            # Dokument-IDs zum Ergebnis hinzufügen
            found_docs.update(inverted_index[word])
    return list(found_docs)

In [52]:
# Testen
search_results = search("Passwort zurücksetzen")
print(f"Suchergebnisse: {search_results}")

Suchergebnisse: [0, 1, 2, 3]


In [53]:
# Bonus: Verbesserte Suche mit Ranking : Die Ergebnisse werden nach Trefferanzahl sortiert
def ranking_search(query):
    # Preprocessing der Suchanfrage
    query_words = preprocess(query)
    # Trefferzähler initialisieren
    doc_scores = {}
    for word in query_words:
        if word in inverted_index:
            for doc_id in inverted_index[word]:
                if doc_id not in doc_scores:
                    doc_scores[doc_id] = 0
                doc_scores[doc_id] += 1
    # Dokumente nach Trefferanzahl sortieren
    ranked_docs = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
    return [doc_id for doc_id, score in ranked_docs]

In [54]:
def rank_search(query):
    query_words = preprocess(query)
    # Score-Dictionary: Schlüssel = Dokument-ID, Wert = Trefferanzahl
    scores = {}
    for word in query_words:
        if word in inverted_index:
            for doc_id in inverted_index[word]:
                if doc_id not in scores:
                    scores[doc_id] = 0
                scores[doc_id] += 1
    # Dokumente nach Score sortieren
    ranked_results = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    return ranked_results # Gibt Liste von (doc_id, score) zurück

In [55]:
# Testen
search_results = ranking_search("Passwort zurücksetzen")
print(f"Suchergebnisse mit Ranking: {search_results}")



results = rank_search("Passwort zurücksetzen")

for doc_id, score in results:
    print(f"[Score: {score}] {documents[doc_id]}")

Suchergebnisse mit Ranking: [0, 2, 1, 3]
[Score: 3] Wie kann ich mein Passwort Passwort zurücksetzen?
[Score: 2] Passwort vergessen - wie kann ich es zurücksetzen?
[Score: 1] Wie ändere ich mein Passwort?
[Score: 1] Wie kann ich mein Login-Passwort erneuern?


## 2. **Gewichtete Suche** mit Whoosh

Bei dieser Aufgabe lernen Sie die IR-Library `Whoosh` kennen. 

Wir bauen mit `Whoosh` einen Index, fügen Dokumente hinzu und führen dann eine Suche aus. 

`Whoosh` ermöglicht es uns, eine gewichtete Suche zu machen, z.B. mit `TF-IDF` oder `BM25`.

In [56]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
from whoosh import scoring  # for choosing the ranking algorithm
import os, shutil

# Schema für den Index definieren
schema = Schema(
    id=ID(stored=True),       # Dokument-ID
    content=TEXT(stored=True)  # Dokument-Text
)

# Verzeichnis für den Index vorbereiten
if os.path.exists('indexdir'):
    shutil.rmtree('indexdir')
os.mkdir('indexdir')

# Index erzeugen und Dokumente hinzufügen
ix = create_in('indexdir', schema)
writer = ix.writer()

PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'indexdir\\MAIN_57juswgq5r2qsyo3.seg'

**2.1 Aufgabe:** Vervollständigen Sie den folgenden Code, indem Sie die `???` ersetzen. 

In [ ]:
# Documente hinzufügen
for i, doc in enumerate(documents):
    writer.add_document(id=str(i), content=doc)
writer.commit()

**2.2.** Aufgabe: 

- Testen Sie die folgende Suchfunktion mit verschiedenen Queries. 
- Verändern Sie das Limit der angezeigten Suchergebnisse. 
- Bei Bedarf können Sie auch den Beispielkorpus `documents` verändern. #neue Sätze hinzufügen


In [ ]:
# Durchsuchen des Index mit TF-IDF Gewichtung -> seltene Wörter haben mehr Gewicht
query = 'wie lange' 

with ix.searcher(weighting=scoring.TF_IDF()) as searcher:  
    query_str = query
    parser = QueryParser('content', ix.schema)
    query = parser.parse(query_str)

    # Index durchsuchen, top n Treffer anzeigen
    top_n = 5
    results = searcher.search(query, limit=top_n)

    print("TF-IDF Ranking:")
    for hit in results:
        print(f"Score: {hit.score:.3f} | Doc {hit['id']}: {hit['content']}")


TF-IDF Ranking:
Score: 5.137 | Doc 29: Wie lange dauert der Versand?
Score: 5.137 | Doc 30: Versanddauer - wie lange dauert es?


In [ ]:
# neue query und anzahl der Treffer
query = 'bestellung' 

with ix.searcher(weighting=scoring.TF_IDF()) as searcher:  
    query_str = query
    parser = QueryParser('content', ix.schema)
    query = parser.parse(query_str)

    # Index durchsuchen, top n Treffer anzeigen
    top_n = 7
    results = searcher.search(query, limit=top_n)

    print("TF-IDF Ranking:")
    for hit in results:
        print(f"Score: {hit.score:.3f} | Doc {hit['id']}: {hit['content']}")


TF-IDF Ranking:
Score: 2.099 | Doc 24: Wie kann ich meine Bestellung stornieren?
Score: 2.099 | Doc 25: Bestellung rückgängig machen wie?
Score: 2.099 | Doc 26: Wie annulliere ich eine Bestellung?
Score: 2.099 | Doc 27: Kann ich meine Bestellung noch stornieren?
Score: 2.099 | Doc 28: Stornierung einer Bestellung Anleitung
Score: 2.099 | Doc 31: Wann wird meine Bestellung geliefert?
Score: 2.099 | Doc 33: Wie schnell kommt meine Bestellung an?


**Aufgabe 2.3:** 

- Passen Sie den Code so an, dass `BM25F` für die Gewichtung verwendet wird.
- Probieren Sie die Suche erneut aus und schauen Sie, ob sich etwas verändert. 
- Passen Sie auch hier ggf. den Beispielkorpus an, z.B. indem Sie längere Dokumente hinzufügen. 

In [ ]:
# Lösung: 

#Suche mit BM25 Gewichtung -> bessere Ergebnisse als TF-IDF
query = 'bestellung'
with ix.searcher(weighting=scoring.BM25F()) as searcher:
    query_str = query
    parser = QueryParser('content', ix.schema)
    query = parser.parse(query_str)

    # Index durchsuchen, top n Treffer anzeigen
    top_n = 5
    results = searcher.search(query, limit=top_n)

    print("BM25 Ranking:")
    for hit in results:
        print(f"Score: {hit.score:.3f} | Doc {hit['id']}: {hit['content']}")


BM25 Ranking:
Score: 2.349 | Doc 25: Bestellung rückgängig machen wie?
Score: 2.349 | Doc 28: Stornierung einer Bestellung Anleitung
Score: 2.349 | Doc 37: Bestellung nachträglich ändern - möglich?
Score: 2.166 | Doc 26: Wie annulliere ich eine Bestellung?
Score: 2.166 | Doc 31: Wann wird meine Bestellung geliefert?


**Aufgabe 2.4:**

- Vergleichen Sie die Suchergebnisse der manuellen Suche mit den Ergebnissen der gewichteten Suche. 
- Gibt es Unterschiede zwischen den Ergebnissen basierend auf TF-IDF vs. BM25F?

Antwort: Reihenfolge wird geändert, längere Dokumente mit relevanten Begriffen könnten nun höher gerankt werden.

## 3. **Semantische Suche** mit Sentence Transformers

- In dieser Demo nutzen Sie die `Sentence Transformers` Library, um Embeddings für den Beispielkorpus zu erzeugen. 
- Diese Embeddings nutzen Sie, um semantisch ähnliche Dokumente zu Ihrer Suche zu finden, mit Hilfe der Kosinus-Ähnlichkeit. 

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Laden eines vortrainierten Modells
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embeddings für die Dokumente erzeugen
doc_embeddings = model.encode(documents, convert_to_tensor=True)

c:\Users\lsfed\Documents\HTW\LLM_KI\rag_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Funktion für semantische Suche, basierend auf Kosinus-Ähnlichkeit
def semantic_search(query, top_k=3):
    """
    Performs semantic search using sentence-transformers built-in cosine similarity.
    
    Returns top_k documents and their similarity scores.
    """
    # Embedding für die Query erzeugen
    query_emb = model.encode(query, convert_to_tensor=True)
    
    # Kosinus-Ähnlichkeit zwischen Query und allen Dokumenten berechnen
    cosine_scores = util.cos_sim(query_emb, doc_embeddings)  # shape: (1, num_docs)
    
    # Ranking der Dokumente, nach höchstem Ähnlichkeits-Score; begrenzt durch Parameter top_k
    ranked_indices = cosine_scores[0].argsort(descending=True)[:top_k]
    
    # Rückgabe der ähnlichsten Dokumente
    return [(int(idx), float(cosine_scores[0][idx])) for idx in ranked_indices]

In [ ]:
# Testen
queries = ['bestellung', 'rückgabe', 'versand']

for q in queries:
    print(f'\nQuery: {q}')
    for idx, score in semantic_search(q):
        print(f'  Score: {score:.3f} | Doc {idx}: {documents[idx]}')


Query: bestellung
  Score: 0.737 | Doc 33: Wie schnell kommt meine Bestellung an?
  Score: 0.663 | Doc 25: Bestellung rückgängig machen wie?
  Score: 0.650 | Doc 31: Wann wird meine Bestellung geliefert?

Query: rückgabe
  Score: 0.413 | Doc 25: Bestellung rückgängig machen wie?
  Score: 0.257 | Doc 22: Akzeptierte Zahlungsmethoden - Übersicht
  Score: 0.255 | Doc 17: Wo kann ich den Kundenservice erreichen?

Query: versand
  Score: 0.472 | Doc 30: Versanddauer - wie lange dauert es?
  Score: 0.463 | Doc 11: Wie ändere ich meine Versandadresse?
  Score: 0.456 | Doc 29: Wie lange dauert der Versand?


## 4. Hausaufgabe: Ein vortrainiertes **Modell für deutschsprachige Dokumente** finden

Ihre Aufgabe ist es, ein vortrainiertes Modell zu finden, das sich für die semantische Suche auf Deutsch eignet. 


Hierfür haben Sie zwei Möglichkeiten: 

**Multilinguale Modelle:** Viele Modelle sind multilingual, z. B. distiluse-base-multilingual-cased-v2, welches Deutsch unterstützt. Multilinguale Modelle sind flexibel für viele Sprachen, eventuell mit leicht geringerer Genauigkeit für Deutsch.

**Deutsch-spezifische Modelle:** Einige Modelle wurden speziell auf deutsche Texte feinjustiert. Deutsch-spezifische Modelle bieten oft eine höhere Genauigkeit für deutsche Texte, eignen sich aber nicht für andere Sprachen.

### Schritt 1: Hugging Face Modelle durchsuchen

- Besuchen Sie: https://huggingface.co/models

- Verwenden Sie die Suchleiste mit Keywords wie z.B.:

„german sentence-transformer“

„multilingual sentence embeddings“
- Alternativ können Sie auch den Filter der Suchfunktion verwenden, um auf die Library `sentence-transformers` und deutsche Sprache zu fokussieren. 

- Wählen Sie ein Modell aus. 

- Prüfen Sie die Beschreibung und stellen Sie sicher, dass Deutsch (`de`) unterstützt wird.


### Schritt 2: Modell anwenden

- Passen Sie den obigen Code so an, dass das von Ihnen ausgewählte Modell verwendet wird. 

- Probieren Sie einige Suchanfragen erneut aus. 

- Vergleichen Sie das Ergebnis mit der vorherigen Suche. 

#### Cross English & German RoBERTa for Sentence Embeddings



In [59]:
from sentence_transformers import SentenceTransformer

# Modell laden
model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
# documents = sehe oben
# Embeddings erzeugen
embeddings = model.encode(documents, convert_to_tensor=True)



No sentence-transformers model found with name T-Systems-onsite/cross-en-de-roberta-sentence-transformer. Creating a new one with mean pooling.


In [61]:
def semantic_search(query, top_k=3):
    query_emb = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_emb, embeddings)
    top_results = sorted(enumerate(cosine_scores[0]), key=lambda x: x[1], reverse=True)[:top_k]
    return [(idx, float(score)) for idx, score in top_results]

In [62]:
# Testen
queries = ['bestellung', 'rückgabe', 'versand']

for q in queries:
    print(f'\nQuery: {q}')
    for idx, score in semantic_search(q):
        print(f'  Score: {score:.3f} | Doc {idx}: {documents[idx]}')


Query: bestellung
  Score: 0.260 | Doc 22: Akzeptierte Zahlungsmethoden - Übersicht
  Score: 0.172 | Doc 28: Stornierung einer Bestellung Anleitung
  Score: 0.140 | Doc 21: Wie kann ich bezahlen?

Query: rückgabe
  Score: 0.345 | Doc 25: Bestellung rückgängig machen wie?
  Score: 0.315 | Doc 2: Passwort vergessen - wie kann ich es zurücksetzen?
  Score: 0.266 | Doc 28: Stornierung einer Bestellung Anleitung

Query: versand
  Score: 0.140 | Doc 28: Stornierung einer Bestellung Anleitung
  Score: 0.116 | Doc 21: Wie kann ich bezahlen?
  Score: 0.101 | Doc 20: Welche Bezahlmöglichkeiten gibt es?
